# Fitness function

> Notebook contenente idee di fitness functions

## Versione MAX
$$\mathbf{Fitness} = W_{acc} \cdot \text{Acc} + W_{struct} \cdot \text{ROUGE} - W_{tok} \cdot \text{Token} - W_{time} \cdot \text{Time} $$


La funzione di valutazione combina quattro componenti fondamentali:

- **Accuracy (Acc)**: Misura la correttezza del contenuto (ottenuta tramite Keyword Matching).

- **ROUGE-L (ROUGE)**: Misura la somiglianza strutturale con la risposta di riferimento.

- **Token Count (Token)**: Misura la lunghezza della risposta.

- **Latency (Time)**: Misura il tempo impiegato per generare la risposta.


## Versione MIN

$$\text{Loss} = W_{err} \cdot (1 - \text{Acc}) + W_{struct} \cdot (1 - \text{ROUGE}) + W_{tok} \cdot \text{TokenCount} + W_{time} \cdot \text{Latency}$$

# Versione "Problema di Massimizzazione"

In [1]:
# Installa la libreria 'dss' di Hugging Face
!pip install datasets=="2.19.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


In [6]:
from datasets import load_dataset

ds = load_dataset("kaist-ai/CoT-Collection")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for kaist-ai/CoT-Collection contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kaist-ai/CoT-Collection
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datase

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# 4. Visualizzazione della struttura del ds
print("\n--- Struttura del dataset ---")
print(ds)

# 5. Visualizzazione dei primi esempi del set di addestramento (train)
# Le colonne importanti sono 'source' (domanda) e 'target' (risposta finale)
# e 'rationale' (Chain-of-Thought, il ragionamento aperto)
print("\n--- Primi Esempi (Split 'train') ---")

# Modificato per prendere più esempi
examples = ds['train'].select(range(5))

for item in examples:
    print(f"Domanda: {item['source']}\nTarget: {item['target']}\nRationale: {item['rationale']}\n")
    print("="*60)


--- Struttura del dataset ---
DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'rationale', 'task', 'type'],
        num_rows: 1837928
    })
})

--- Primi Esempi (Split 'train') ---
Domanda: Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow tree

# Testing fitness function

In [8]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1eb46aaf67186b626b0b81c44b1cd00bd67ac7f15618bea68265424d54917a38
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


### Fitness function class

In [9]:
import evaluate
import numpy as np

# Carichiamo la metrica ROUGE se non è già caricata
try:
    rouge_metric
except NameError:
    rouge_metric = evaluate.load("rouge")

class FitnessCalculator:
    def __init__(self,
                 w_accuracy=2.0,       # Ex Alpha: Quanto conta la risposta corretta
                 w_token_penalty=0.02, # Ex Beta: Costo per ogni parola usata
                 w_latency_penalty=0.1,# Ex Gamma: Costo per ogni secondo impiegato
                 w_rouge_reward=1.0):  # Ex Delta: Premio per la somiglianza strutturale

        self.w_acc = w_accuracy
        self.w_token = w_token_penalty
        self.w_time = w_latency_penalty
        self.w_rouge = w_rouge_reward

    def calculate_keyword_accuracy(self, prediction, reference):
        """
        Verifica se le parole chiave del target sono presenti nella predizione.
        Ritorna 1.0 se tutte le parole non-stopword sono presenti, altrimenti 0.0.
        """
        pred_words = set(prediction.strip().lower().split())
        ref_words = set(reference.strip().lower().split())

        # Stop words base per evitare falsi positivi grammaticali
        stop_words = {'the', 'a', 'an', 'in', 'on', 'at', 'to', 'from', 'of', 'and', 'is', 'are'}
        important_ref_words = ref_words - stop_words

        # Gestione casi limite (target vuoto o solo stopwords)
        if not important_ref_words:
            return 1.0 if reference.strip().lower() in prediction.strip().lower() else 0.0

        # Calcolo copertura
        matches = len(important_ref_words.intersection(pred_words))
        coverage = matches / len(important_ref_words)

        return 1.0 if coverage == 1.0 else 0.0

    def compute(self, generated_text, target_text, generation_time):
        """
        Calcola il punteggio di fitness totale.
        """
        # 1. Accuracy (Correttezza semantica di base)
        accuracy = self.calculate_keyword_accuracy(generated_text, target_text)

        # 2. ROUGE (Somiglianza strutturale)
        rouge_results = rouge_metric.compute(predictions=[generated_text], references=[target_text])
        rouge_l = rouge_results['rougeL']

        # 3. Conteggio Token (Lunghezza)
        token_count = len(generated_text.split())

        # --- LOGICA DI GATING ---
        # Se la risposta è sbagliata (accuracy 0), ignoriamo il premio ROUGE.
        # Non vogliamo premiare belle frasi che dicono cose false.
        effective_rouge = rouge_l if accuracy > 0 else 0.0

        # --- CALCOLO FITNESS ---
        # Fitness = (Premio Accuratezza) + (Premio Struttura) - (Costo Lunghezza) - (Costo Tempo)
        fitness = (self.w_acc * accuracy) + \
                  (self.w_rouge * effective_rouge) - \
                  (self.w_token * token_count) - \
                  (self.w_time * generation_time)

        return {
            "fitness": round(fitness, 4),
            "details": {
                "accuracy_score": accuracy,
                "rouge_score": round(effective_rouge, 4),
                "tokens_used": token_count,
                "time_taken": round(generation_time, 4)
            }
        }

# Istanziazione con i nomi chiari
fitness_engine = FitnessCalculator(
    w_accuracy=2.0,
    w_token_penalty=0.02,
    w_latency_penalty=0.1,
    w_rouge_reward=1.0
)

print("Fitness Engine v3.0 pronto con nomenclatura esplicita.")

Fitness Engine v3.0 pronto con nomenclatura esplicita.


### Testing function with some examples

In [10]:
# Dati presi dal tuo dataset
target_real = "from plants"

# Simuliamo 3 diversi scenari di generazione del modello

# SCENARIO A: Risposta perfetta e veloce
gen_a = "They come from plants"
time_a = 0.1 # secondi

# SCENARIO B: Risposta corretta ma troppo prolissa (Verbosa) e lenta
gen_b = "Based on the text provided, we can clearly see that many medicines and recreational drugs actually originate directly from various types of plants found in nature."
time_b = 0.8 # secondi

# SCENARIO C: Risposta sbagliata e breve
gen_c = "from chemicals"
time_c = 0.1 # secondi

# Calcoliamo le fitness
res_a = fitness_calc.compute(gen_a, target_real, time_a)
res_b = fitness_calc.compute(gen_b, target_real, time_b)
res_c = fitness_calc.compute(gen_c, target_real, time_c)

# Funzione per stampare bene i risultati
def print_report(name, text, res):
    print(f"--- {name} ---")
    print(f"Testo: '{text}'")
    print(f"FITNESS TOTALE: {res['total_fitness']}")
    print(f"Dettagli: Accuracy: {res['components']['accuracy']} | "
          f"Rouge Reward: +{res['components']['rouge_reward']} | "
          f"Token Penalty: -{res['components']['token_penalty']} | "
          f"Latency Penalty: -{res['components']['latency_penalty']}")
    print("\n")

print_report("Scenario A (Conciso e Corretto)", gen_a, res_a)
print_report("Scenario B (Prolisso)", gen_b, res_b)
print_report("Scenario C (Sbagliato)", gen_c, res_c)

NameError: name 'fitness_calc' is not defined

ok test passato ✅

### TEST 2 Fine-grained test

In [ ]:
import pandas as pd

# Definiamo il target per questo test
target_ref = "Battle of Appomattox Court House"

# Definiamo una lista di casi limite (Edge Cases)
test_cases = [
    # 1. PERFETTO
    {"desc": "Perfetta", "text": "Battle of Appomattox Court House", "time": 0.1},

    # 2. VARIAZIONI CORRETTE
    {"desc": "Corretta (Minuscolo)", "text": "battle of appomattox court house", "time": 0.1},
    {"desc": "Corretta + Intro (Verbose)", "text": "The correct answer is the Battle of Appomattox Court House", "time": 0.5},
    {"desc": "Corretta + Intro Lunghissima", "text": "I have analyzed the history books and I can confidently say that the event in question is definitely the Battle of Appomattox Court House which ended the war.", "time": 1.2},

    # 3. CASI "QUASI" CORRETTI (Testiamo la severità della keyword accuracy)
    {"desc": "Manca 1 parola (Battle)", "text": "Appomattox Court House", "time": 0.1},
    {"desc": "Manca 1 parola (House)", "text": "Battle of Appomattox Court", "time": 0.1},

    # 4. HALLUCINAZIONI (Parole simili ma fatto diverso)
    {"desc": "Hallucination (Gettysburg)", "text": "Battle of Gettysburg Court House", "time": 0.2},

    # 5. CASI SBAGLIATI
    {"desc": "Sbagliata corta", "text": "Civil War", "time": 0.1},
    {"desc": "Sbagliata lunga", "text": "General Lee surrendered at a different location entirely.", "time": 0.5},

    # 6. RUMORE
    {"desc": "Vuota/Simboli", "text": "...", "time": 0.01},
]

results = []

# Eseguiamo il test
print(f"TARGET: '{target_ref}'\n")

for case in test_cases:
    # Calcoliamo la fitness
    score = fitness_engine.compute(case["text"], target_ref, case["time"])

    # Salviamo i dati per la tabella
    results.append({
        "Descrizione": case["desc"],
        "Risposta Generata": case["text"],
        "Fitness": score["fitness"],
        "Acc (Kwd)": score["details"]["accuracy_score"],
        "Rouge": score["details"]["rouge_score"],
        "Tokens": score["details"]["tokens_used"],
        "Time": score["details"]["time_taken"]
    })

# Creiamo un DataFrame per visualizzare bene
df_results = pd.DataFrame(results)

# Ordiniamo per Fitness decrescente (i migliori in alto)
df_results = df_results.sort_values(by="Fitness", ascending=False)

# Visualizzazione formattata
from IPython.display import display
display(df_results)

# Versione 2: Fitness "Minimizzabile"


In [20]:
import evaluate
import numpy as np

# Carichiamo ROUGE se necessario
try:
    rouge_metric
except NameError:
    rouge_metric = evaluate.load("rouge")

class CostCalculator:
    def __init__(self,
                 w_accuracy=10.0,        # Penalità ENORME per risposta sbagliata
                 w_token_cost=0.05,   # Costo per ogni parola usata
                 w_time_cost=0.5,     # Costo per ogni secondo impiegato
                 w_divergence=2.0):   # Costo per struttura diversa dal target

        self.w_err = w_accuracy
        self.w_tok = w_token_cost
        self.w_time = w_time_cost
        self.w_div = w_divergence

    def calculate_keyword_accuracy(self, prediction, reference):
        """
        Ritorna 1.0 se corretto, 0.0 se sbagliato.
        """
        pred_words = set(prediction.strip().lower().split())
        ref_words = set(reference.strip().lower().split())
        stop_words = {'the', 'a', 'an', 'in', 'on', 'at', 'to', 'from', 'of', 'and', 'is', 'are'}
        important_ref_words = ref_words - stop_words

        if not important_ref_words:
            return 1.0 if reference.strip().lower() in prediction.strip().lower() else 0.0

        matches = len(important_ref_words.intersection(pred_words))
        coverage = matches / len(important_ref_words)
        return 1.0 if coverage == 1.0 else 0.0

    def compute(self, generated_text, target_text, generation_time):
        """
        Calcola la LOSS totale (Minimizzare).
        Obiettivo teorico: 0.0 (Impossibile, ma ci tendiamo).
        """
        # 1. Accuracy
        accuracy = self.calculate_keyword_accuracy(generated_text, target_text)
        # Trasformiamo in Errore (0 se corretto, 1 se sbagliato)
        error_rate = 1.0 - accuracy

        # 2. ROUGE (Struttura)
        rouge_results = rouge_metric.compute(predictions=[generated_text], references=[target_text])
        rouge_l = rouge_results['rougeL']

        # GATING: Se la risposta è sbagliata (Error=1), non ci interessa se la struttura è simile.
        # Consideriamo la divergenza massima (1.0) o calcoliamo la divergenza reale solo se corretto.
        # Qui: Calcoliamo la "Divergenza" (1 - ROUGE).
        # Se accuracy è 0, la divergenza non aiuta a recuperare punti, l'errore domina.
        structural_divergence = 1.0 - rouge_l

        # 3. Costi di Risorse
        token_count = len(generated_text.split())

        # --- CALCOLO LOSS ---

        # Costo Base = (Peso Errore * Errore) + (Peso Divergenza * Divergenza)
        # Se accuracy = 1 (corretto), il primo termine sparisce.
        base_cost = (self.w_err * error_rate) + (self.w_div * structural_divergence)

        # Costo Efficienza = Token + Tempo
        efficiency_cost = (self.w_tok * token_count) + (self.w_time * generation_time)

        total_loss = base_cost + efficiency_cost

        return {
            "loss": round(total_loss, 4),
            "details": {
                "is_correct": accuracy > 0,
                "error_cost": round(self.w_err * error_rate, 4),
                "divergence_cost": round(self.w_div * structural_divergence, 4),
                "token_cost": round(self.w_tok * token_count, 4),
                "time_cost": round(self.w_time * generation_time, 4)
            }
        }

# Inizializzazione
# Nota come w_accuracy (10.0) è molto più alto di w_token_cost * lunghezza_media
loss_engine = CostCalculator(
    w_accuracy=10.0,       # Priorità assoluta: NON SBAGLIARE
    w_token_cost=0.05,  # Preferisci risposte brevi
    w_time_cost=0.5,    # Preferisci risposte veloci
    w_divergence=1.0    # Preferisci struttura simile
)

print("Cost Function (Loss) pronta per la minimizzazione.")

Cost Function (Loss) pronta per la minimizzazione.


### Test

In [21]:
import pandas as pd

target_ref = "Battle of Appomattox Court House"

test_cases = [
    # IL VINCITORE ATTESO (Basso costo)
    {"desc": "Perfetta", "text": "Battle of Appomattox Court House", "time": 0.1},

    # COSTO MEDIO (Corretto ma costoso)
    {"desc": "Corretta ma Lunga", "text": "The correct answer is the Battle of Appomattox Court House", "time": 0.5},

    # COSTO ALTO (Sbagliato)
    {"desc": "Sbagliata corta", "text": "Civil War", "time": 0.1},

    # COSTO ALTISSIMO (Sbagliato + Lungo)
    {"desc": "Sbagliata lunga", "text": "General Lee surrendered at a different location entirely.", "time": 0.5},

    # TRAPPOLA (Vuota/Veloce)
    {"desc": "Vuota (Empty Trap)", "text": ".", "time": 0.01},
]

results = []

print(f"TARGET: '{target_ref}'\n")

for case in test_cases:
    res = loss_engine.compute(case["text"], target_ref, case["time"])

    results.append({
        "Descrizione": case["desc"],
        "LOSS (Minimizzare)": res["loss"], # Più basso = Meglio
        "Corretto?": res["details"]["is_correct"],
        "Err Cost": res["details"]["error_cost"],
        "Div Cost": res["details"]["divergence_cost"],
        "Tok Cost": res["details"]["token_cost"]
    })

df_loss = pd.DataFrame(results)
# Ordiniamo dal più basso (migliore) al più alto
df_loss = df_loss.sort_values(by="LOSS (Minimizzare)", ascending=True)

from IPython.display import display
display(df_loss)

TARGET: 'Battle of Appomattox Court House'



,Descrizione,LOSS (Minimizzare),Corretto?,Err Cost,Div Cost,Tok Cost
0,Perfetta,0.3000,True,0.0,0.0000,0.25
1,Corretta ma Lunga,1.0833,True,0.0,0.3333,0.50
4,Vuota (Empty Trap),11.0550,False,10.0,1.0000,0.05
2,Sbagliata corta,11.1500,False,10.0,1.0000,0.10
3,Sbagliata lunga,11.6500,False,10.0,1.0000,0.40


# Versione 3: Includiamo anche la valutazione del "rationale"

Includendo il razionale abbiamo una metrica aggiuntiva che aiuta a prevenire le "lucky-guesses" certificando anche la motivazione dietro la risposta data.
Tuttavia, utilizzare ROUGE-L potrebbe non essere ideale, dato che:
1. lo stesso ragionamento può essere espresso strutturando la frase in infiniti modi;
2. in secondo luogo, una frase corta a più probabilità di ricevere uno score basso dalla metrica, e.g. "per via della pioggia" e "poiche pioveva" otterrebbe un punteggio basso in questo caso dato non ci sono parole "overlappate" tuttavia il ragionamento è corretto.

✅ **Soluzione:** utiliziamo SBERT (Sentence Bert), modello basato su Semantic Embedding, che permette di trasformare la frase in un embedding che codifica la semantica della frase. Possiamo quindi fare l'encoding del ragionamento fornito dall'agente e compararlo con quello reference.



In [11]:
!pip install sentence-transformers evaluate rouge_score

In [27]:
import evaluate
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Carichiamo i modelli una volta sola (Singleton pattern per performance)
try:
    rouge_metric
except NameError:
    rouge_metric = evaluate.load("rouge")

try:
    sbert_model # usiamo il modello già caricato se la cella è gia stata eseguita
except NameError:
    print("Caricamento modello SBERT (potrebbe richiedere qualche secondo)...")
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("SBERT caricato.")

class CostCalculatorSBERT:
    def __init__(self,
                 w_accuracy=2.0,       # Priorità assoluta all'accuratezza della risposta
                 w_rationale=2.0,      # Peso ALTO per la coerenza semantica del ragionamento
                 w_token_cost=0.05,    # Costo verbosità risposta
                 w_time_cost=0.5,      # Costo latenza
                 w_rouge_ans=1.0):     # Similarità strutturale risposta breve

        self.w_acc = w_accuracy
        self.w_rat = w_rationale
        self.w_tok = w_token_cost
        self.w_time = w_time_cost
        self.w_rouge_ans = w_rouge_ans
        self.model = sbert_model

    def _calculate_keyword_accuracy(self, prediction, reference):
        """Keyword matching per la risposta breve."""
        pred_words = set(prediction.strip().lower().split())
        ref_words = set(reference.strip().lower().split())
        stop_words = {'the', 'a', 'an', 'in', 'on', 'at', 'to', 'from', 'of', 'and', 'is', 'are', 'by'}
        important_ref_words = ref_words - stop_words

        if not important_ref_words:
            return 1.0 if reference.strip().lower() in prediction.strip().lower() else 0.0

        matches = len(important_ref_words.intersection(pred_words))
        coverage = matches / len(important_ref_words)
        return 1.0 if coverage == 1.0 else 0.0

    def _calculate_semantic_loss(self, gen_text, ref_text):
        """
        Calcola la Loss basata sulla Cosine Similarity di SBERT.
        Similarity va da -1 (opposto) a 1 (identico).
        Loss = 1 - Similarity.
        Range Loss: 0 (perfetto) a 2 (completamente diverso).
        """
        if not gen_text.strip():
            return 1.0 # Penalità default per testo vuoto

        # Encoding
        embeddings1 = self.model.encode(gen_text, convert_to_tensor=True)
        embeddings2 = self.model.encode(ref_text, convert_to_tensor=True)

        # Cosine Similarity
        cosine_score = util.pytorch_cos_sim(embeddings1, embeddings2).item()

        # Trasformiamo in Loss (Minimizzare)
        # Se score è 1.0 (identico) -> Loss 0.0
        # Se score è 0.0 (non c'entra nulla) -> Loss 1.0
        return 1.0 - cosine_score

    def compute(self, gen_ans, target_ans, gen_rat, target_rat, generation_time):

        # --- 1. RISPOSTA BREVE (Answer) ---
        accuracy = self._calculate_keyword_accuracy(gen_ans, target_ans)
        rouge_ans = rouge_metric.compute(predictions=[gen_ans], references=[target_ans])['rougeL']

        # Costi Risposta
        error_cost = (1.0 - accuracy) * self.w_acc * 5.0 # Penalità errore brutale
        ans_div_cost = (1.0 - rouge_ans) * self.w_rouge_ans
        token_cost = len(gen_ans.split()) * self.w_tok

        # --- 2. RAGIONAMENTO (Rationale) - Powered by SBERT ---
        # Qui usiamo SBERT invece di ROUGE
        semantic_divergence = self._calculate_semantic_loss(gen_rat, target_rat)
        rat_cost = semantic_divergence * self.w_rat

        # --- 3. TOTALE ---
        total_loss = error_cost + ans_div_cost + rat_cost + token_cost + (self.w_time * generation_time)

        return {
            "loss": round(total_loss, 4),
            "details": {
                "is_correct": accuracy > 0,
                "error_cost": round(error_cost, 4),
                "rat_semantic_loss": round(rat_cost, 4), # Costo semantico SBERT
                "raw_similarity": round(1.0 - semantic_divergence, 4), # Quanto si somigliano (0-1)
                "ans_struct_loss": round(ans_div_cost, 4),
                "token_loss": round(token_cost, 4)
            }
        }

engine_sbert = CostCalculatorSBERT()
print("Engine SBERT pronto.")

Engine SBERT pronto.


## Test (SBERT vs ROUGE-L)

In [23]:
import evaluate
import numpy as np
import pandas as pd

# Carichiamo ROUGE (se non è già caricato)
try:
    rouge_metric
except NameError:
    rouge_metric = evaluate.load("rouge")

class CostCalculatorROUGEL:
    def __init__(self,
                 w_accuracy=2.0,       # Peso critico: Penalità se la risposta è sbagliata
                 w_rationale=1.5,      # NUOVO: Peso per la qualità del ragionamento
                 w_token_cost=0.05,    # Costo per ogni parola nella risposta finale
                 w_time_cost=0.5,      # Costo per ogni secondo impiegato
                 w_rouge_ans=1.0):     # Peso per la struttura della risposta finale

        self.w_acc = w_accuracy
        self.w_rat = w_rationale
        self.w_tok = w_token_cost
        self.w_time = w_time_cost
        self.w_rouge_ans = w_rouge_ans

    def _calculate_keyword_accuracy(self, prediction, reference):
        """
        Verifica se le parole chiave del target sono presenti nella predizione.
        Return: 1.0 (Corretto) o 0.0 (Sbagliato)
        """
        pred_words = set(prediction.strip().lower().split())
        ref_words = set(reference.strip().lower().split())
        stop_words = {'the', 'a', 'an', 'in', 'on', 'at', 'to', 'from', 'of', 'and', 'is', 'are'}
        important_ref_words = ref_words - stop_words

        if not important_ref_words:
            return 1.0 if reference.strip().lower() in prediction.strip().lower() else 0.0

        matches = len(important_ref_words.intersection(pred_words))
        coverage = matches / len(important_ref_words)
        return 1.0 if coverage == 1.0 else 0.0

    def compute(self, gen_ans, target_ans, gen_rat, target_rat, generation_time):
        """
        Calcola la LOSS (Minimizzazione).
        Obiettivo: 0.0 (O il più basso possibile).
        """

        # --- 1. VALUTAZIONE RISPOSTA (ANSWER) ---
        # Accuratezza (0 o 1)
        accuracy = self._calculate_keyword_accuracy(gen_ans, target_ans)

        # ROUGE Risposta (Similarità strutturale)
        rouge_ans_res = rouge_metric.compute(predictions=[gen_ans], references=[target_ans])
        rouge_ans_l = rouge_ans_res['rougeL']

        # --- CALCOLO COSTI RISPOSTA ---
        # Costo Errore: Se sbaglia, penalità enorme (es. 5 volte il peso base)
        error_cost = (1.0 - accuracy) * self.w_acc * 5.0

        # Costo Divergenza Risposta: (1 - ROUGE). Se sono uguali, costo 0.
        ans_divergence_cost = (1.0 - rouge_ans_l) * self.w_rouge_ans

        # Costo Token (Lunghezza della risposta finale)
        token_cost = len(gen_ans.split()) * self.w_tok

        # --- 2. VALUTAZIONE RAGIONAMENTO (RATIONALE) ---
        # Calcoliamo ROUGE sul Rationale
        if not gen_rat.strip():
            # Se non ha generato ragionamento, penalità massima
            rat_divergence_cost = self.w_rat * 1.0
        else:
            rouge_rat_res = rouge_metric.compute(predictions=[gen_rat], references=[target_rat])
            rouge_rat_l = rouge_rat_res['rougeL']
            # Costo Divergenza: Quanto il ragionamento si discosta dal target
            rat_divergence_cost = (1.0 - rouge_rat_l) * self.w_rat

        # --- 3. SOMMA TOTALE (LOSS) ---
        total_loss = error_cost + \
                     ans_divergence_cost + \
                     rat_divergence_cost + \
                     token_cost + \
                     (self.w_time * generation_time)

        return {
            "loss": round(total_loss, 4),
            "details": {
                "is_correct": accuracy > 0,
                "error_cost_ans": round(error_cost, 4),
                "div_cost_ans": round(ans_divergence_cost, 4),
                "div_cost_rat": round(rat_divergence_cost, 4), # <--- Il costo del rationale
                "token_cost": round(token_cost, 4),
                "time_cost": round(self.w_time * generation_time, 4)
            }
        }

# Inizializzazione Engine
engine = CostCalculatorROUGEL()
print("Engine v4 pronto (Minimizzazione con Rationale).")

Engine v4 pronto (Minimizzazione con Rationale).


In [31]:
import pandas as pd

# --- CONFIGURAZIONE PESI (Identici per confronto equo) ---
weights = {
    "w_accuracy": 2.0,
    "w_rationale": 2.0,  # Peso importante sul ragionamento
    "w_token_cost": 0.05,
    "w_time_cost": 0.5,
    "w_rouge_ans": 1.0
}

# Istanziamo i giudici
judge_rouge = CostCalculatorROUGEL(**weights)        # Vecchio (Word Overlap)
judge_sbert = CostCalculatorSBERT(**weights)   # Nuovo (Semantic Embedding)

# --- DATASET REALE (I tuoi 4 esempi) ---
dataset_samples = [
    # ESEMPIO 1: Phytochemistry
    {
        "id": "Phyto",
        "tgt_ans": "from plants",
        "tgt_rat": """The article states that many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. These are some examples of the medicines found in plants mentioned by the author. Thus it can be stated with certainty that some medicines do indeed come from plants. Therefore, "from plants" is the correct answer option to this question based on the context provided.""",
        # SIMULAZIONE AGENTE BUONO (Parafrasi sintetica)
        "good_rat": "Based on the text, drugs like morphine and caffeine originate directly from plants. Therefore, the correct answer is that they come from plants.",
        # SIMULAZIONE AGENTE CATTIVO (Parole chiave presenti, senso rotto)
        "bad_rat": "Plants are recreational drugs and medicine states that author found caffeine in the context provided."
    },
    # ESEMPIO 2: Legal Acts
    {
        "id": "Legal",
        "tgt_ans": "Regulation",
        "tgt_rat": """This act has all the features of Regulation. The act is addressed to an European Union country; it must be applied once and for all within a certain time-limit; it lays down general rules of application, which are binding on all Member States.""",
        # SIMULAZIONE AGENTE BUONO (Logica corretta)
        "good_rat": "It identifies as a Regulation because it is binding in its entirety and directly applicable to all Member States as described in the text.",
        # SIMULAZIONE AGENTE CATTIVO (Generico / Hallucination)
        "bad_rat": "I chose this category because the text talks about European Union laws and taxes."
    },
    # ESEMPIO 3: Civil War
    {
        "id": "CivilWar",
        "tgt_ans": "Battle of Appomattox Court House",
        "tgt_rat": """The Battle of Appomattox Court House was a battle in the final stages of the American Civil War, resulting in Confederate General Robert E. Lee surrendering his Army to Union Commander Ulysses S. Grant on April 9th 1865""",
        # SIMULAZIONE AGENTE BUONO
        "good_rat": "General Lee surrendered to Grant at the Appomattox Court House, marking the end of the Civil War in 1865.",
        # SIMULAZIONE AGENTE CATTIVO (Vero ma irrilevante per la domanda specifica)
        "bad_rat": "The American Civil War was a conflict between the Union and the Confederacy involving General Lee and Grant."
    },
    # ESEMPIO 4: Nasser Heart Attacks
    {
        "id": "Nasser",
        "tgt_ans": "major heart attacks",
        "tgt_rat": """The answer is the number of major heart attacks. Though he was on bed rest for six weeks after his second attack, that's not greater than the total number of times (i.e., two) Nasser suffered from a heart attack.""",
        # SIMULAZIONE AGENTE BUONO (Ragionamento matematico corretto)
        "good_rat": "Nasser had 2 heart attacks. He spent 6 weeks in bed. Since 2 is smaller than 6, the number of heart attacks is the smaller value.",
        # SIMULAZIONE AGENTE CATTIVO (Logica inversa/sbagliata)
        "bad_rat": "He was on bed rest for six weeks which is a smaller number than the two heart attacks he suffered."
    }
]

results = []

print("--- AVVIO TEST COMPARATIVO SU DATI REALI ---\n")

for sample in dataset_samples:
    # Test Caso BUONO
    res_rouge_good = judge_rouge.compute(sample['tgt_ans'], sample['tgt_ans'], sample['good_rat'], sample['tgt_rat'], 0.5)
    res_sbert_good = judge_sbert.compute(sample['tgt_ans'], sample['tgt_ans'], sample['good_rat'], sample['tgt_rat'], 0.5)

    # Test Caso CATTIVO (Risposta giusta, ragionamento sbagliato)
    res_rouge_bad = judge_rouge.compute(sample['tgt_ans'], sample['tgt_ans'], sample['bad_rat'], sample['tgt_rat'], 0.5)
    res_sbert_bad = judge_sbert.compute(sample['tgt_ans'], sample['tgt_ans'], sample['bad_rat'], sample['tgt_rat'], 0.5)

    results.append({
        "ID": sample['id'],
        "Type": "GOOD (Coherent)",
        "Rat Cost (ROUGE)": res_rouge_good['details']['div_cost_rat'],
        "Rat Cost (SBERT)": res_sbert_good['details']['rat_semantic_loss'],
        "Winner": "SBERT" if res_sbert_good['details']['rat_semantic_loss'] < res_rouge_good['details']['div_cost_rat'] else "ROUGE",
        "Note": "-"
    })

    results.append({
        "ID": sample['id'],
        "Type": "BAD (Incoherent)",
        "Rat Cost (ROUGE)": res_rouge_bad['details']['div_cost_rat'],
        "Rat Cost (SBERT)": res_sbert_bad['details']['rat_semantic_loss'],
        "Note": "SBERT dovrebbe penalizzare di più"
    })

df_real = pd.DataFrame(results)

from IPython.display import display
display(df_real)

--- AVVIO TEST COMPARATIVO SU DATI REALI ---



,ID,Type,Rat Cost (ROUGE),Rat Cost (SBERT),Winner,Note
0,Phyto,GOOD (Coherent),1.4783,0.3955,SBERT,-
1,Phyto,BAD (Incoherent),1.6190,0.5760,NaN,SBERT dovrebbe penalizzare di più
2,Legal,GOOD (Coherent),1.5882,0.6456,SBERT,-
3,Legal,BAD (Incoherent),1.6552,1.0794,NaN,SBERT dovrebbe penalizzare di più
4,CivilWar,GOOD (Coherent),1.2281,0.3121,SBERT,-
5,CivilWar,BAD (Incoherent),1.4286,0.9114,NaN,SBERT dovrebbe penalizzare di più
6,Nasser,GOOD (Coherent),1.4627,0.2248,SBERT,-
7,Nasser,BAD (Incoherent),1.2131,0.7046,NaN,SBERT dovrebbe penalizzare di più
